In [1]:

"""
使用lm-format-enforcer实现结构化信息抽取。
支持： transformers, LangChain, LlamaIndex, llama.cpp, vLLM, Haystack, NVIDIA TensorRT-LLM, ExLlamaV2.
"""




from pydantic import BaseModel, Field
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn
from transformers import pipeline

from typing import Annotated, List




In [2]:

class NameLocation(BaseModel):

    names: Annotated[List[str], Field(description="抽取所有的姓名")]
    citys: Annotated[List[str], Field(description="抽取所有的城市")]




# class AnswerFormat(BaseModel):
#     first_name: str
#     last_name: str
#     year_of_birth: int
#     num_seasons_in_nba: int

# Create a transformers pipeline
hf_pipeline = pipeline('text-generation', model='/root/autodl-tmp/modelscope_cache/hub/Qwen/Qwen2-7B-Instruct-AWQ', 
                       device_map='auto',max_length=512)
prompt = f'输入文本：张三和李四都喜欢上海。\n 待抽取的json schema: {NameLocation.schema_json()} :\n'

# Create a character level parser and build a transformers prefix function from it
parser = JsonSchemaParser(NameLocation.schema())
prefix_function = build_transformers_prefix_allowed_tokens_fn(hf_pipeline.tokenizer, parser)

# Call the pipeline with the prefix function
output_dict = hf_pipeline(prompt, prefix_allowed_tokens_fn=prefix_function)

# Extract the results
result = output_dict[0]['generated_text'][len(prompt):]
print(result)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{
"names": ["张三", "李四"],
"citys": ["上海"]
}
